In [1]:
import pandas as pd
import geopandas as gpd
import re
import matplotlib as plt
import numpy as np

In [2]:
trash = pd.read_csv('cleaned_trash_haulers.csv')

In [3]:
trash['Date Opened'] = pd.to_datetime(trash['Date Opened'])

In [4]:
trash['Zip Code'] = trash['Zip Code'].astype('str')
trash['Zip Code'] = trash['Zip Code'].str.extract(r'(\d{5})')

In [5]:
trash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20226 entries, 0 to 20225
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Request Number  20226 non-null  int64         
 1   Date Opened     20226 non-null  datetime64[ns]
 2   Request         20226 non-null  object        
 3   Description     20192 non-null  object        
 4   Zip Code        20186 non-null  object        
 5   Trash Hauler    19364 non-null  object        
 6   Trash Route     19279 non-null  object        
 7   LONGITUDE       20198 non-null  float64       
 8   LATITUDE        20198 non-null  float64       
 9   Clean Address   20216 non-null  object        
 10  Missed          20226 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 1.6+ MB


# Red River Waste Solutions Total Damages
- For each address:
    - `>= 3 missed pickups on 3 unique dates`
    - `Within a 6 month timespan`
    + `+ $1500`

In [6]:
redriver = trash[['Request Number', 'Date Opened', 'Missed', 'Clean Address', 'Trash Hauler']]

In [7]:
redriver.head()

,Request Number,Date Opened,Missed,Clean Address,Trash Hauler
0,25270,2017-11-01,False,3817 CROUCH DR,RED RIVER
1,25274,2017-11-01,True,4028 CLARKSVILLE PIKE,RED RIVER
2,25276,2017-11-01,True,6528 THUNDERBIRD DR,RED RIVER
3,25307,2017-11-01,True,2603 OLD MATTHEWS RD,WASTE IND
4,25312,2017-11-01,True,604 CROLEY DR,RED RIVER


In [8]:
redriver = redriver.loc[redriver['Trash Hauler'] == 'RED RIVER']

In [9]:
redriver = redriver.loc[redriver['Missed'] == True]

In [10]:
redriver.head()

,Request Number,Date Opened,Missed,Clean Address,Trash Hauler
1,25274,2017-11-01,True,4028 CLARKSVILLE PIKE,RED RIVER
2,25276,2017-11-01,True,6528 THUNDERBIRD DR,RED RIVER
4,25312,2017-11-01,True,604 CROLEY DR,RED RIVER
8,25330,2017-11-01,True,4484 LAVERGNE COUCHVILLE PIKE,RED RIVER
10,25341,2017-11-01,True,3113 HYDES FERRY RD,RED RIVER


In [11]:
redriver.head()

,Request Number,Date Opened,Missed,Clean Address,Trash Hauler
1,25274,2017-11-01,True,4028 CLARKSVILLE PIKE,RED RIVER
2,25276,2017-11-01,True,6528 THUNDERBIRD DR,RED RIVER
4,25312,2017-11-01,True,604 CROLEY DR,RED RIVER
8,25330,2017-11-01,True,4484 LAVERGNE COUCHVILLE PIKE,RED RIVER
10,25341,2017-11-01,True,3113 HYDES FERRY RD,RED RIVER


In [12]:
redriver = redriver.drop_duplicates(['Date Opened', 'Clean Address'])

In [13]:
redriver.sort_values(['Date Opened', 'Clean Address']).reset_index(drop = True)

,Request Number,Date Opened,Missed,Clean Address,Trash Hauler
0,25474,2017-11-01,True,1144 SEWANEE RD,RED RIVER
1,25469,2017-11-01,True,117 LAFAYETTE CT,RED RIVER
2,25529,2017-11-01,True,131 PAGE RD,RED RIVER
3,25525,2017-11-01,True,162 ANTIOCH PIKE,RED RIVER
4,25496,2017-11-01,True,1815 WOODMONT BLVD,RED RIVER
...,...,...,...,...,...
12666,266662,2019-11-01,True,655 HERITAGE DR,RED RIVER
12667,266915,2019-11-01,True,675 HERITAGE DR,RED RIVER
12668,266945,2019-11-01,True,801 E CAMPBELL RD,RED RIVER
12669,266862,2019-11-01,True,816 E CAMPBELL RD,RED RIVER


In [29]:
redriverrolled = redriver.groupby('Clean Address').rolling(window = '180d', on = 'Date Opened').count()

In [32]:
redriverrolled.sort_values()
redriverrolled

AttributeError: 'DataFrame' object has no attribute 'sort'